<a href="https://colab.research.google.com/github/YosyValero/jvalero59_TFG/blob/main/Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baseline TFG

### Se diseña un modelo base como primera parte del Trabajo Fin de Grado (TFG). El trabajo versa sobre el Procesamiento de Lenguaje Natural y en concreto en el Reconocimiento de Entidades Nombradas (NER)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [13]:
import numpy as np
import pandas as pd

In [14]:
path_tsv = '/content/drive/MyDrive/Dataset/tsv/'

In [15]:
df_dissea =  pd.read_csv(path_tsv + 'dissea.tsv', sep='\t', header=0).loc[:, ('tweets_id', 'type')].rename(columns={'tweets_id': 'tweet_id'})

In [16]:
df_prof =  pd.read_csv(path_tsv + 'prof.tsv', sep='\t', header=0).loc[:, ('tweet_id', 'type')]

In [17]:
df_prof.drop(df_prof[df_prof['type']=='-'].index, inplace=True)

In [18]:
df_prof = df_prof.replace(['PROFESION', 'SITUACION_LABORAL', 'ACTIVIDAD', 'FIGURATIVA'], ['NO_ENFERMEDAD', 'NO_ENFERMEDAD', 'NO_ENFERMEDAD', 'NO_ENFERMEDAD'])

In [19]:
df_prof_norep = df_prof.drop_duplicates(ignore_index=True)

In [20]:
df_disse_norep = df_dissea.drop_duplicates(ignore_index=True)

In [21]:
muestras_min = min([len(df_disse_norep), len(df_prof_norep)])

In [22]:
muestras = 50

In [23]:
if muestras_min < muestras:
  muestras = muestras_min

In [24]:
df_xamples_diss = df_disse_norep.head(muestras)

In [25]:
df_xamples_nodiss = df_prof_norep.head(muestras)

In [26]:
df_xamples_nodiss = df_xamples_nodiss.rename(index=lambda x: x + muestras)

In [27]:
df_tot = pd.concat([df_xamples_diss, df_xamples_nodiss])


In [28]:
df_tot

,tweet_id,type
0,1357198223706894339,ENFERMEDAD
1,1494382407398662147,ENFERMEDAD
2,1478289344565755910,ENFERMEDAD
3,1476287203424800781,ENFERMEDAD
4,1269536913259671559,ENFERMEDAD
...,...,...
95,1242946691428495365,NO_ENFERMEDAD
96,1252570761312829440,NO_ENFERMEDAD
97,1251429071705247744,NO_ENFERMEDAD
98,1288737608160419840,NO_ENFERMEDAD


In [29]:
df_tot.insert(2, 'content_tuit', value='')

In [30]:
def get_tuit(type_tuit, name_tuit):
  sub_path = path_tsv + 'train-valid-txt-files/'
  if type_tuit == 'ENFERMEDAD':
    path_tuit = sub_path + 'dissea'
  else:
    path_tuit = sub_path + 'prof'

  path_tuit = path_tuit + '/training/' + name_tuit + '.txt'
  tuit = open(path_tuit, mode='r', encoding='utf-8')
  content_tuit = tuit.read()
  tuit.close()

  return content_tuit

In [31]:
for i in range(len(df_tot)):
    name_tuit = df_tot.loc[i, 'tweet_id']
    type_tuit = df_tot.loc[i, 'type']
    df_tot.at[i, 'content_tuit'] = get_tuit(type_tuit, str(name_tuit))

In [32]:
df_tot

,tweet_id,type,content_tuit
0,1357198223706894339,ENFERMEDAD,"No sé si es alergia, covid o gripe, ya solo es..."
1,1494382407398662147,ENFERMEDAD,ADOLESCENTES CON DIABETES\n\nHola!\nMe llamo E...
2,1478289344565755910,ENFERMEDAD,De niño y adulto me vacunaron contra el saramp...
3,1476287203424800781,ENFERMEDAD,Un ingreso por depresión debería entenderse co...
4,1269536913259671559,ENFERMEDAD,"Santiago, desde Córdoba no me enteré que estuv..."
...,...,...,...
95,1242946691428495365,NO_ENFERMEDAD,"Margarita Salas, bioquímica pionera descubrió ..."
96,1252570761312829440,NO_ENFERMEDAD,"Mientras hace negocio vendiendo mascarillas, p..."
97,1251429071705247744,NO_ENFERMEDAD,Libertad ya!!! Vea el documental de ABC donde...
98,1288737608160419840,NO_ENFERMEDAD,Limitados los aforos en todos los lugares de E...


In [33]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [34]:
symbols = [
    '¿','?','~','`','!','¡','@','$','%','^',
    '*','(',')','_','-','+','=','{','}','[',
    ']','\\',';','<','>','/', '.', ',','&','\n'
    ,'\r','\t', '|', '“', '"', '–', '”', '©', '-', '—',
    '…', ';', ':', '‘','’', '...', '#', '«', '»', '``',
    "''", '..', '...', '--'
]

In [35]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [36]:
def limpia_tuit_sp(text):
    str_limp = ""
    stp_wors_mas_sym = stopwords.words('spanish') + symbols

    # Eliminamos las urls de los tuits
    text = re.sub('http[s]*://[a-zA-Z0-9/.-]*|www.[a-zA-Z0-9/.-]*', '', text)

    # Eliminamos numeros
    text = re.sub('[0-9]*','', text)
    # print(text)

    # Eliminamos emoticonos
    text = remove_emoji(text)

    # Eliminamos otros caracteres
    text = re.sub('[¿,_,?,¡,!,-]*','', text)

    # Tokenizamos los tuits
    text_tok = word_tokenize(text)
    for t_tok in text_tok:
        if not t_tok in stp_wors_mas_sym:
            str_limp = str_limp + " " + t_tok
    str_limp =  str_limp.lower()
    return str_limp

In [37]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### A partir del dataset convertimos los tuits y sus type (ENFERMEDAD, NO_ENFERMEDAD) en 2 arrays

In [38]:
x, y = np.asarray(df_tot['content_tuit']), np.asarray(df_tot['type'])


In [40]:
from sklearn.model_selection import train_test_split

In [41]:
# Troceamos el dataset en crudo con vines didácticos
x_train_raw, x_test_raw, y_train_raw, y_test_raw = train_test_split(x, y, train_size=70, random_state=42)

Limpiamos los tuit

In [42]:
tuits_limpios = [limpia_tuit_sp(t) for t in x]

### Conversión de los datos textuales a númericos, usos:
*   CountVectorizer
*   Tfidf

In [43]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [44]:
count_vectorizer = CountVectorizer()
X_numer_count = count_vectorizer.fit_transform(tuits_limpios).toarray()

In [45]:
tfid_vectorizer = TfidfVectorizer()
X_numer_tfid = tfid_vectorizer.fit_transform(tuits_limpios).toarray()

### Importamos las librerias para implementar el modelo mediante Support Vector Machine y la división del dataset es subset entrenamiento (train) y testeo (test)

In [46]:
from sklearn import svm
clf = svm.SVC(kernel='linear')

SVM_CountVectorized

In [47]:
x_train, x_test, y_train, y_test = train_test_split(X_numer_count, y, train_size=70, random_state=42)
clf.fit(x_train, y_train)

SVC(kernel='linear')

Se va a probar la prección del modelo definido utilizado algunos tuits del subset test

In [48]:
tuits_pruebas_limpios = [limpia_tuit_sp(t) for t in x_test_raw]

In [ ]:
for x_tuit, y_tuit in zip(tuits_pruebas_limpios, y_test_raw):
  print(x_tuit, y_tuit)


In [50]:
tuits_pruebas_count = count_vectorizer.transform(tuits_pruebas_limpios).toarray()

In [ ]:
predicted = clf.predict(tuits_pruebas_count)
for doc, categoria in zip(tuits_pruebas_limpios, predicted):
    print('%r => %s' % (doc, categoria))
print()

# MÉTRICAS

### Score (acurracy)

In [52]:
clf.score(x_test, y_test)

0.7666666666666667

### F1 Score

In [53]:
from sklearn.metrics import f1_score

f1_score(y_test, clf.predict(x_test),
         labels=['ENFERMEDAD','NO_ENFERMEDAD'],
         average=None)

array([0.74074074, 0.78787879])

### Reporte de Clasificación

In [54]:
from sklearn.metrics import classification_report

print(classification_report(y_test, clf.predict(x_test),
         labels=['ENFERMEDAD','NO_ENFERMEDAD']))

               precision    recall  f1-score   support

   ENFERMEDAD       1.00      0.59      0.74        17
NO_ENFERMEDAD       0.65      1.00      0.79        13

     accuracy                           0.77        30
    macro avg       0.82      0.79      0.76        30
 weighted avg       0.85      0.77      0.76        30



In [55]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, clf.predict(x_test),
         labels=['ENFERMEDAD','NO_ENFERMEDAD'])


array([[10,  7],
       [ 0, 13]])

## Optimización (GridSearchCV)

In [ ]:
from sklearn.model_selection import GridSearchCV

parametros = {'C':[1,4,8,16,32], 'kernel':['linear','rbf']}
clf = svm.SVC()
clf_grid = GridSearchCV(clf, parametros, cv=5)
clf_grid.fit(x_train, y_train)


In [57]:
clf_grid.best_score_

0.8

In [58]:
clf_grid.best_params_

{'C': 1, 'kernel': 'rbf'}

SVM_TfidfVectorizer

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_numer_tfid, y, train_size=70, random_state=42)
clf.fit(x_train, y_train)
clf.score(x_test, y_test)

0.5333333333333333

In [ ]:
clf.set_params(kernel='rbf').fit(x_train, y_train)

In [ ]:
clf.score(x_test, y_test)